In [1]:
import os

os.environ["CONFIG_APP_DIR"] = "config"


In [2]:
import sys
import os
import pickle
os.chdir("..")
from omegaconf import OmegaConf


In [18]:
from dataclasses import dataclass
from typing import List


@dataclass
class SourceField:
    source: str
    field: str

@dataclass
class Function:
    target_field: str
    method_name: str
    pattern_method: str
    source: List[SourceField]
    target_collection: str = None
    


flow_config = OmegaConf.load(os.path.join("config", "pipeline", "flow.yaml"))
func_map = {}
for column_transformation in flow_config:
    name = str(column_transformation)
    item = dict(flow_config[column_transformation])
    item['source'] = [SourceField(str(field_type), str(field)) for field_type in item['source'] for field in item['source'][field_type]]
    dict_input = dict(**item, **{'target_field': column_transformation})
    func_map.update({name: Function(**dict_input)})
#     print(name)
#     print(f"pattern_set_up_party_type({flow_config[column_transformation]['source']})")
    

In [103]:
def draw_design(func, field="", nested_level=1):
    tabs = nested_level * '\t'
    if field:
        field = f"[{field}]"
    design = f"{func}{field}"       
    try:
        func = func_map[func]
    except KeyError:
        return design
    design +=  f"\n\t{tabs}<-"
    for field in func.source:
        design += draw_design(field.field, field.source, nested_level+1) + f"\n\t{tabs}"
    return design

In [104]:
print(draw_design('AP_PARTY_TYPE'))

AP_PARTY_TYPE
		<-ALL_PARTY_TYPES[alertedParty]
			<-partyType[parties]
			
		datasetType[metadata]
		PARTY1_ORGANIZATION_NAME[xml_fields]
		ORGANIZATIONPERSONIND[xml_fields]
		supplementalInfo.legalFormName[alertSupplementalInfo]
		WLP_TYPE[watchlistParty]
		
